# Functioning example for the ZIPRA library
This library provides tools to extract and process the .SAFE file containing Sentinel-2 data

In [48]:
import pandas as pd
import numpy as np
from shapely import geometry
import requests
import leafmap as leafmap
from shapely.geometry import shape
import rasterio as rio
from rasterio.plot import show


## How to get the data

For using the library, a fundamental step is to download the zip folder containing the data. 
Below is a script that guides the user to obtain such data using the Odata endpoint provided by Copernicus Data Space Catalogue [Odata info](https://documentation.dataspace.copernicus.eu/APIs/OData.html)

First this is defining an AOI, use the draw control instruments to draw the desired AOI

In [ ]:
# Draw the AOI on the map below
m = leafmap.Map(center=[45.64, 9.60], zoom=5, toolbar_control=False, fullscreen_control=False)
m


Map(center=[45.64, 9.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [ ]:
# This section extract the AOI from the map and extracts its bounding box
AOI = m.draw_features
if not AOI:
    print("Please, select an area of interest on the map")
else:
    AOI = shape(AOI[0]['geometry'])
    bbox = AOI.bounds
    print(f"The Shapely geometry: {AOI}")
    print(f"The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): {bbox}")

The Shapely geometry: POLYGON ((10.17334 45.452424, 10.17334 45.828799, 10.667725 45.828799, 10.667725 45.452424, 10.17334 45.452424))
The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): (10.17334, 45.452424, 10.667725, 45.828799)


In [26]:
# This section allows to define some parameters for searching Sentinel-2 products, 
# if the user already has the ID of the product, they can skip this and the following cell

# Copernicus Data Space Catalogue OData endpoint
catalogue_odata_url = "https://catalogue.dataspace.copernicus.eu/odata/v1"
collection_name = "SENTINEL-2"
product_type = "S2MSI2A"

# USER DEFINED PARAMETERS
# Define the maximum cloud coverage
max_cloud_cover = 30

#Define the period
search_period_start = "2025-09-15T00:00:00.000Z"
search_period_end = "2025-09-30T00:00:00.000Z"
# END OF USER DEFINED PARAMETERS

aoi = geometry.box(*bbox).wkt

In [27]:
search_query = f"{catalogue_odata_url}/Products?$filter=Collection/Name eq '{collection_name}' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{product_type}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and ContentDate/Start gt {search_period_start} and ContentDate/Start lt {search_period_end} and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover})"
response = requests.get(search_query).json()
result = pd.DataFrame.from_dict(response["value"])
result.head(5)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,1c0a5551-71dc-4cef-8ad8-0adf9646eb0a,S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_2...,application/octet-stream,1208045165,2025-09-17T16:46:30.000000Z,2025-09-17T16:50:55.145532Z,2025-09-17T16:52:30.465127Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/17/S2B_MSIL...,"[{'Value': '0511ae0acba4dc85007a09f1bfd188c0',...","{'Start': '2025-09-17T10:20:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((8.99974146026188...,"{'type': 'Polygon', 'coordinates': [[[8.999741..."
1,application/octet-stream,94760a27-d074-4704-bca6-6015505d1a42,S2C_MSIL2A_20250929T100821_N0511_R022_T32TPR_2...,application/octet-stream,1201928935,2025-09-29T16:46:56.000000Z,2025-09-29T16:53:11.041991Z,2025-09-29T16:54:27.989520Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/29/S2C_MSIL...,"[{'Value': '95588eaceca38437e8239060ad074fc9',...","{'Start': '2025-09-29T10:08:21.025000Z', 'End'...",geography'SRID=4326;POLYGON ((10.2925320358333...,"{'type': 'Polygon', 'coordinates': [[[10.29253..."
2,application/octet-stream,5a775652-c561-4a1e-aa07-0c0d3bb2be63,S2A_MSIL2A_20250921T100731_N0511_R022_T32TNR_2...,application/octet-stream,579443327,2025-09-21T13:11:05.000000Z,2025-09-21T13:18:27.459708Z,2025-09-21T13:20:10.192657Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/21/S2A_MSIL...,"[{'Value': 'df73fc82029f92d35f371f327b5edbec',...","{'Start': '2025-09-21T10:07:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((9.64106618865031...,"{'type': 'Polygon', 'coordinates': [[[9.641066..."
3,application/octet-stream,9bb683fb-7ad2-47e4-804c-5c91c0853192,S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_2...,application/octet-stream,598037871,2025-09-29T16:46:47.000000Z,2025-09-29T16:51:57.542905Z,2025-09-29T16:53:24.082903Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/29/S2C_MSIL...,"[{'Value': '422c042439f3100db80c496a33645e61',...","{'Start': '2025-09-29T10:08:21.025000Z', 'End'...",geography'SRID=4326;POLYGON ((9.62737689938526...,"{'type': 'Polygon', 'coordinates': [[[9.627376..."
4,application/octet-stream,377348a3-6b33-4e11-9c9b-c31366a4123d,S2C_MSIL2A_20250919T101041_N0511_R022_T32TNR_2...,application/octet-stream,583590448,2025-09-19T16:41:39.000000Z,2025-09-19T16:46:28.175522Z,2025-09-19T16:48:10.521363Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/19/S2C_MSIL...,"[{'Value': '36584fb427c0f7a49436a67abdb6df3f',...","{'Start': '2025-09-19T10:10:41.025000Z', 'End'...",geography'SRID=4326;POLYGON ((9.64199873485010...,"{'type': 'Polygon', 'coordinates': [[[9.641998..."


In [28]:
# Select the image you want to download

# USER DEFINED PARAMETERS
Index_image = 0  # Change this index to select different images from the search result
path=f"./DATA/"
# END OF USER DEFINED PARAMETERS

Image_Id = result.iloc[Index_image]["Id"]
Image_name = result.iloc[Index_image]["Name"]
url = f"https://download.dataspace.copernicus.eu/odata/v1/Products({Image_Id})/$value"
path = path + f"{Image_name}.zip"

Before proceding with the download, the user should create an account on Copernicus Data Space and get OAuth client, a guide can be found at this [link](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html)

In [ ]:
# Authentication credentials
# USER DEFINED PARAMETERS
username = "INSERT_YOUR_USERNAME"
password = "INSERT_YOUR_PASSWORD"
# END OF USER DEFINED PARAMETERS

In [29]:
username = "sh-0cf3e199-e14a-4820-a512-91ff5b24db26"
password = "BX3pIbsPWALpe52i0R3uAUOSNaVeQh4M"

In [30]:
# Get an access token and establish a session
token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
token_data = {
    "grant_type": "client_credentials",
    "client_id": username,
    "client_secret": password
}
token_response = requests.post(token_url, data=token_data)
access_token = token_response.json().get("access_token")

print("Token response:", token_response.status_code)
session = requests.Session()
session.headers["Authorization"] = f"Bearer {access_token}"

Token response: 200


In [31]:
#Download the image, this might take few minutes
try:
    response = session.get(url, stream=True)
    response.raise_for_status()
    zip_data = response.content
    response.close()

    # Writes the downloaded content to a file
    with open(path, 'wb') as f:
        f.write(zip_data)

    print(f"\nData saved to '{path}' (Size: {len(zip_data)} bytes).")

except requests.exceptions.RequestException as e:
    print(f"\nAn error occurred during the request: {e}")


Data saved to './DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE.zip' (Size: 1208045165 bytes).


## Zipra library example

In [32]:
from ZIPRA import Band_estraction
tiff_file, band_list = Band_estraction(path)

File ZIP decompressed successfully.
band folder: ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRANULE/L2A_T32TNR_A044564_20250917T102153/IMG_DATA
Band B02 found at ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRANULE/L2A_T32TNR_A044564_20250917T102153/IMG_DATA/R10m/T32TNR_20250917T102019_B02_10m.jp2
Band B03 found at ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRANULE/L2A_T32TNR_A044564_20250917T102153/IMG_DATA/R10m/T32TNR_20250917T102019_B03_10m.jp2
Band B04 found at ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRANULE/L2A_T32TNR_A044564_20250917T102153/IMG_DATA/R10m/T32TNR_20250917T102019_B04_10m.jp2
Band B08 found at ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRANULE/L2A_T32TNR_A044564_20250917T102153/IMG_DATA/R10m/T32TNR_20250917T102019_B08_10m.jp2
Band B12 found at ./DATA/S2B_MSIL2A_20250917T102019_N0511_R065_T32TNR_20250917T155807.SAFE/GRA